# प्राणी तज्ज्ञ प्रणाली अंमलात आणणे

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) मधील एक उदाहरण.

या नमुन्यात, आपण काही शारीरिक वैशिष्ट्यांवर आधारित प्राणी ओळखण्यासाठी एक साधी ज्ञान-आधारित प्रणाली अंमलात आणू. ही प्रणाली खालील AND-OR झाडाद्वारे दर्शविली जाऊ शकते (हे संपूर्ण झाडाचा एक भाग आहे, आपण सहजपणे आणखी काही नियम जोडू शकतो):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.mr.png)


## मागील अनुमानासह आमची स्वतःची तज्ज्ञ प्रणाली शेल

चला उत्पादन नियमांवर आधारित ज्ञानाचे प्रतिनिधित्व करण्यासाठी एक साधी भाषा परिभाषित करण्याचा प्रयत्न करूया. नियम परिभाषित करण्यासाठी आम्ही Python वर्गांचा कीवर्ड म्हणून वापर करू. मुख्यतः 3 प्रकारचे वर्ग असतील:
* `Ask` वापरकर्त्याला विचारायच्या प्रश्नाचे प्रतिनिधित्व करते. यात संभाव्य उत्तरांचा संच असतो.
* `If` नियमाचे प्रतिनिधित्व करते, आणि नियमाच्या सामग्री साठवण्यासाठी फक्त एक सिंटॅक्टिक शुगर आहे.
* `AND`/`OR` हे वर्ग झाडाच्या AND/OR शाखांचे प्रतिनिधित्व करण्यासाठी आहेत. ते फक्त आतल्या युक्तिवादांची यादी साठवतात. कोड सुलभ करण्यासाठी, सर्व कार्यक्षमता पालक वर्ग `Content` मध्ये परिभाषित केली आहे.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

आमच्या प्रणालीमध्ये, कार्यरत स्मृतीमध्ये **गुणधर्म-मूल्य जोड्या** म्हणून **तथ्यांची** यादी असेल. ज्ञानकोशाला एक मोठा शब्दकोश म्हणून परिभाषित करता येईल, जो क्रिया (कार्यरत स्मृतीमध्ये समाविष्ट करावयाच्या नवीन तथ्ये) अटींशी नकाशित करतो, ज्या AND-OR अभिव्यक्ती म्हणून व्यक्त केल्या जातात. तसेच, काही तथ्ये `विचारली` जाऊ शकतात.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

बॅकवर्ड इनफरन्स करण्यासाठी, आपण `Knowledgebase` नावाचा वर्ग परिभाषित करू. हा वर्ग खालील गोष्टींचा समावेश करेल:
* कार्यरत `memory` - एक डिक्शनरी जी गुणधर्मांना त्यांच्या मूल्यांशी नकाशित करते
* `rules` - वरीलप्रमाणे परिभाषित केलेल्या स्वरूपातील ज्ञानभांडार नियम

मुख्य दोन पद्धती आहेत:
* `get` - एखाद्या गुणधर्माचे मूल्य प्राप्त करण्यासाठी, आवश्यक असल्यास इनफरन्स करणे. उदाहरणार्थ, `get('color')` हे रंगाच्या स्लॉटचे मूल्य मिळवेल (जर आवश्यक असेल तर विचारेल आणि नंतरच्या वापरासाठी कार्यरत मेमरीमध्ये मूल्य साठवेल). जर आपण `get('color:blue')` विचारले, तर ते रंग विचारेल आणि नंतर रंगावर आधारित `y`/`n` मूल्य परत करेल.
* `eval` - प्रत्यक्ष इनफरन्स करते, म्हणजे AND/OR ट्री ट्रॅव्हर्स करते, उप-लक्ष्यांचे मूल्यांकन करते, इत्यादी.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

आता आपण आपले प्राणी ज्ञानकोश परिभाषित करूया आणि सल्लामसलत करूया. लक्षात ठेवा की ही प्रक्रिया तुम्हाला प्रश्न विचारेल. तुम्ही `y`/`n` टाइप करून होय-नाही प्रश्नांची उत्तरे देऊ शकता, किंवा लांब बहुपर्यायी उत्तरांसाठी संख्या (0..N) निर्दिष्ट करून उत्तर देऊ शकता.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## PyKnow वापरून फॉरवर्ड इनफरन्स

पुढील उदाहरणात, ज्ञान सादरीकरणासाठीच्या लायब्ररींपैकी एक, [PyKnow](https://github.com/buguroo/pyknow/) वापरून फॉरवर्ड इनफरन्स कसे अंमलात आणायचे ते पाहू. **PyKnow** ही Python मध्ये फॉरवर्ड इनफरन्स सिस्टीम तयार करण्यासाठीची लायब्ररी आहे, जी जुन्या पारंपरिक प्रणाली [CLIPS](http://www.clipsrules.net/index.html) सारखी डिझाइन केली गेली आहे.

आपण स्वतः फॉरवर्ड चेनिंग अंमलात आणू शकलो असतो, आणि त्यात फारसे अडचणीही आल्या नसत्या, पण साध्या अंमलबजावणी सहसा फार कार्यक्षम नसतात. अधिक प्रभावी नियम जुळवण्यासाठी एक विशेष अल्गोरिदम [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) वापरला जातो.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

आम्ही आमची प्रणाली `KnowledgeEngine` चे उपवर्ग असलेल्या वर्ग म्हणून परिभाषित करू. प्रत्येक नियम `@Rule` या अॅनोटेशनसह स्वतंत्र फंक्शनद्वारे परिभाषित केला जातो, जो नियम कधी कार्यान्वित होईल हे निर्दिष्ट करतो. नियमाच्या आत, आम्ही `declare` फंक्शनचा वापर करून नवीन तथ्ये जोडू शकतो, आणि ती तथ्ये जोडल्याने पुढील अनुमान इंजिनद्वारे आणखी काही नियम कार्यान्वित केले जातील.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

एकदा आपण ज्ञानकोश परिभाषित केला की, आपण आपल्या कार्यरत स्मृतीत काही प्रारंभिक तथ्ये भरतो आणि नंतर `run()` पद्धत कॉल करून अनुमान प्रक्रिया करतो. परिणामी, आपण पाहू शकता की नवीन अनुमानित तथ्ये कार्यरत स्मृतीत जोडली जातात, ज्यामध्ये प्राण्याबद्दलचे अंतिम तथ्य समाविष्ट आहे (जर आपण सर्व प्रारंभिक तथ्ये योग्यरित्या सेट केली असतील तर).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील मूळ दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
